# Chatbot summarization with postgress

In [1]:
%pip install --quiet -U langchain_groq langchain_core langgraph langgraph-checkpoint-postgres psycopg psycopg-pool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.3 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
apikey = userdata.get('GROQ_API_KEY')

In [3]:
from langchain_groq import ChatGroq
llm = ChatGroq(model= 'llama-3.3-70b-versatile', temperature = 0, api_key=apikey)

In [4]:
from google.colab import userdata
DB_URI = userdata.get('DB_URI')

In [5]:
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver

# Connection pool for efficient database access
connection_kwargs = {"autocommit": True, "prepare_threshold": 0}

# Create a persistent connection pool
pool = ConnectionPool(conninfo=DB_URI, max_size=20, kwargs=connection_kwargs)

# Initialize PostgresSaver checkpointer
checkpointer = PostgresSaver(pool)
checkpointer.setup()  # Ensure database tables are set up

In [6]:
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

from langgraph.graph import END
from langgraph.graph import MessagesState

class State(MessagesState):
    summary: str

# Define the logic to call the model
def call_model(state: State) -> State:

    # Get summary if it exists
    summary = state.get("summary", "")
    print(f"Using summary: {summary}")

    # If there is summary, then we add it
    if summary:

        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]

    else:
        messages = state["messages"]

    response = llm.invoke(messages)
    return {"messages": response}

def summarize_conversation(state: State) -> State:
    print(f"Messages before summarizing: {len(state['messages'])}")
    # First, we get any existing summary
    summary = state.get("summary", "")
    print(f"Existing summary: {summary}")

    # Create our summarization prompt
    if summary:

        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )

    else:
        summary_message = "Create a summary of the conversation above:"


    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = llm.invoke(messages)
    # Summarization logic
    print(f"New summary: {response.content}")

    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]

    print(f"Messages after truncation: {len(delete_messages)}")
    return {"summary": response.content, "messages": delete_messages}

# Determine whether to end or summarize the conversation
def should_continue(state: State) -> State:

    """Return the next node to execute."""

    messages = state["messages"]
    print(f"Message count: {len(messages)}")
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"

    # Otherwise we can just end
    return END

In [7]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph

# Redefine workflow
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile the workflow with PostgreSQL checkpointer
graph = workflow.compile(checkpointer=checkpointer)

In [8]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="hi! I'm Zeeshan")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 2
================================== Ai Message ==================================

Hello Zeeshan! It's nice to meet you. Is there something I can help you with or would you like to chat?


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Zeeshan", additional_kwargs={}, response_metadata={}, id='dbe3ffe1-05fe-452f-bd8c-1a2d7a1e07ec'), AIMessage(content="Hello Zeeshan! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 42, 'total_tokens': 70, 'completion_time': 0.101818182, 'prompt_time': 0.004619813, 'queue_time': 0.28877038200000005, 'total_time': 0.106437995}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_76dc6cf67d', 'finish_reason': 'stop', 'logprobs': None}, id='run-07ecc975-b88b-4357-9136-bb0a7d7505e3-0', usage_metadata={'input_tokens': 42, 'output_tokens': 28, 'total_tokens': 70})]}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff99ec-e274-658a-8001-c2934318d38c'}}, metadata={'step': 1, 'source': 'loop', 'writes': {'conversation': {'messages': A

In [9]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="I like coding.")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 4
================================== Ai Message ==================================

Coding can be a lot of fun. What kind of coding do you enjoy? Are you into web development, mobile app development, or something else? Do you have a favorite programming language?


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Zeeshan", additional_kwargs={}, response_metadata={}, id='dbe3ffe1-05fe-452f-bd8c-1a2d7a1e07ec'), AIMessage(content="Hello Zeeshan! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 42, 'total_tokens': 70, 'completion_time': 0.101818182, 'prompt_time': 0.004619813, 'queue_time': 0.28877038200000005, 'total_time': 0.106437995}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_76dc6cf67d', 'finish_reason': 'stop', 'logprobs': None}, id='run-07ecc975-b88b-4357-9136-bb0a7d7505e3-0', usage_metadata={'input_tokens': 42, 'output_tokens': 28, 'total_tokens': 70}), HumanMessage(content='I like coding.', additional_kwargs={}, response_metadata={}, id='fac4f585-3d63-4e36-8ad1-2d01a80c5517'), AIMessage(content='Coding can be a lot of fun. What kind of coding do you enjoy? Are yo

In [10]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="What's my name and what is my hobby?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 6
================================== Ai Message ==================================

Your name is Zeeshan, and your hobby is coding.


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Zeeshan", additional_kwargs={}, response_metadata={}, id='dbe3ffe1-05fe-452f-bd8c-1a2d7a1e07ec'), AIMessage(content="Hello Zeeshan! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 42, 'total_tokens': 70, 'completion_time': 0.101818182, 'prompt_time': 0.004619813, 'queue_time': 0.28877038200000005, 'total_time': 0.106437995}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_76dc6cf67d', 'finish_reason': 'stop', 'logprobs': None}, id='run-07ecc975-b88b-4357-9136-bb0a7d7505e3-0', usage_metadata={'input_tokens': 42, 'output_tokens': 28, 'total_tokens': 70}), HumanMessage(content='I like coding.', additional_kwargs={}, response_metadata={}, id='fac4f585-3d63-4e36-8ad1-2d01a80c5517'), AIMessage(content='Coding can be a lot of fun. What kind of coding do you enjoy? Are yo